In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.optimize import curve_fit
import random
import fista
import timeit
from itertools import combinations
from scipy.stats import gaussian_kde
from matplotlib.colors import LogNorm

In [3]:
num_trials = 100
batch_size = 100
pixels = 16

In [4]:
IMAGES = scipy.io.loadmat('IMAGES.mat')
IMAGES = IMAGES['IMAGES']
(imsize, imsize, num_images) = np.shape(IMAGES)

In [5]:
def sample_images(sample_size, IMAGES):
    border = 4  #buffer around the edge of an entire image
    I = np.zeros((pixels**2,sample_size))

    for i in range(sample_size):
        r = border + np.ceil((imsize-pixels-2*border) * random.uniform(0, 1))
        c = border + np.ceil((imsize-pixels-2*border) * random.uniform(0, 1))
        image = IMAGES[int(r):int(r+pixels), int(c):int(c+pixels), int(imi = np.ceil(num_images * random.uniform(0, 1))-1)]
        I[:,i] = np.reshape(image, pixels**2) 
        
    return I

In [ ]:
def activities(I,Q,W,theta):
    return 0

In [ ]:
for t in range(num_trials):
    I = sample_images(batch_size, IMAGES)
    Y = activities(I,Q,W,theta)
    muy = Y.mean(axis = 0)
    Cyy = Y*Y.conj().transpose()/batch_size
    
    dW = alpha*(Cyy - p**2)
    W += dW
    W += -np.diag(np.diag(W))
    W[W < 0] = 0 #force weights to be inhibitory
    dtheta = gamma*(np.sum(acts))
    
    #extract images at random from IMAGES array

In [ ]:
% SAILnet: Sparse And Independent Local network _/)
% Joel Zylberberg
% UC Berkeley Redwood Center for Theoretical Neuroscience
% joelz@berkeley.edu
% Dec 2010
%*****************************************************
% for work stemming from use of this code, please cite
% Zylberberg, Murphy & DeWeese (2011) "A sparse coding model with synaptically
% local plasticity and spiking neurons can account for the diverse shapes of V1
% simple cell receptive fields", PLoS Computational Biology 7(10).
%****************************************************

for t = 1:num_trials

	% extract subimages at random from IMAGES array to make data array X
	for i=1:batch_size
		r=BUFF+ceil((imsize-sz-2*BUFF)*rand);
		c=BUFF+ceil((imsize-sz-2*BUFF)*rand);
		myimage=reshape(IMAGES(r:r+sz-1,c:c+sz-1,ceil(num_images*rand)),N,1);
		myimage = myimage-mean(myimage);
		myimage = myimage/std(myimage);
		X(:,i) = myimage;
    end
    
    % compute network outputs for this data array
    Y = activities(X,Q,W,theta); %gives the spike counts 
    
	% compute statistics for this batch
    muy=mean(Y,2);          %average neuron activities
    Cyy=Y*Y'/batch_size;	%activity correlation between neurons
    
  % update lateral weights with Foldiak's rule: inhibition to maintain decorrelation
   dW = alpha*(Cyy - p^2); 
   W= W + dW; 
   W=W-diag(diag(W));       % zero out diagonal
   W(find(W<0))=0;          % require lateral connections to inhibit
	
	% update feedforward weights with Oja's rule
	square_act = sum((Y.*Y)');
	mymat = diag(square_act);
	dQ =  beta*Y*X'/batch_size - beta*mymat*Q/batch_size; 
	Q=Q + dQ;

	% update thresholds with Foldiak's rule: keep each neuron firing near target
	dtheta =  gamma*(sum(Y')'/batch_size - p); 
	theta=theta + dtheta; 
    
    % compute moving averages of muy and Cyy
    Y_ave=(1-eta_ave)*Y_ave + eta_ave*muy;
    Cyy_ave=(1-eta_ave)*Cyy_ave + eta_ave*Cyy;
    
    % display network state and activity statistics every so often
	if mod(t,50)==0,																	
        figure(1)
        clf
		show_network(theta,Y_ave,W,Cyy_ave,Q);
        figure(2)
        showrfs(Q)
		t		
       
    if mod(t,50000) ==0,
        save current_net
    end    
    end
end

save final_network


In [ ]:
% initialize SAILnet parameters
% Joel Zylberberg, UC Berkeley 2010
% joelz@berkeley.edu
%*****************************************************
% for work stemming from use of this code, please cite
% Zylberberg, Murphy & DeWeese (2011) "A sparse coding model with synaptically
% local plasticity and spiking neurons can account for the diverse shapes of V1
% simple cell receptive fields", PLoS Computational Biology 7(10).
%****************************************************

fprintf('Here we go _/) \n');

clear all

% the whitened image array
load IMAGES

%general params
batch_size =100;        
num_trials = 25000;

%input data
[imsize imsize num_images]=size(IMAGES);
BUFF=20;

% number of inputs
N=256;      %number of pixels in the patches
sz = sqrt(N);

% number of outputs
OC = 1;		%overcompleteness of neurons wrt pixels
M=OC*N;

% target output firing rate; spikes per neuron per image
p=0.05;

% Initialize network parameters 
%feedforward weights Q
%horizontal connections W
%thresholds theta

Q=randn(M,N);
Q=diag(1./sqrt(sum(Q.*Q,2)))*Q;
W=zeros(M);
theta=2*ones(M,1);

% learning rates
alpha=1;
beta=0.01;
gamma=0.1;

% rate parameter for computing moving averages to get activity stats
eta_ave=0.3;

% initialize average activity stats
Y_ave=p;
Cyy_ave=p^2;

In [ ]:
% Joel Zylberberg, UC Berkeley 2010
% joelz@berkeley.edu
% activities.m - Simulate LIF neurons to get spike counts
% function Y=activities(X,Q,W,theta)
% X:        input array
% Q:        feedforward weights
% W:        horizontal weights
% theta:    thresholds
% Y:        outputs
%*****************************************************
% for work stemming from use of this code, please cite
% Zylberberg, Murphy & DeWeese (2011) "A sparse coding model with synaptically
% local plasticity and spiking neurons can account for the diverse shapes of V1
% simple cell receptive fields", PLoS Computational Biology 7(10).
%****************************************************

function Y=activities(X,Q,W,theta)

% size of data array
[N batch_size]=size(X);
sz=sqrt(N);

% number of outputs
M=size(Q,1);

% number iterations
num_iterations=50;

% rate parameter for numerical integration
eta=0.1;

% projections of stimuli onto FF weights
B=Q*X;

% threshold
T=repmat(theta,1,batch_size);

% initialize values
Ys= zeros(M,batch_size);
as= zeros(M,batch_size);
Y = zeros(M,batch_size);

for t=1:num_iterations
	% diffeq for y
    Ys=(1-eta)*Ys + eta*(B - W*as);

	%hard threshold: spikes
	as = ones(M,batch_size);
	as(find(Ys<T)) = 0;
	
    %ad spikes to the count
    Y = Y + as;
    
	%if you spiked, start back at resting potential
	Ys(find(Ys>T)) = 0.0;
	
end

